# Proyecto AI 2

## Clasificación de la causa del accidente cerebro vascular

### Preprocesamiento de imágenes

Para poder realizar el aprendizaje es necesario hacer una preprocesamiento para adquirir la información importante de las imágenes así como poder clasificarlas.

In [8]:
from IPython.display import Image
import os
import pandas as pd
import numpy as np
import cv2
import tifffile as tifi
from PIL import Image
import seam_carving
import time
import csv
import random

In [31]:
f_train = pd.read_csv('train.csv', sep=',')
f_train

,image_id,center_id,patient_id,image_num,label
0,006388_0,11,006388,0,CE
1,008e5c_0,11,008e5c,0,CE
2,00c058_0,11,00c058,0,LAA
3,01adc5_0,11,01adc5,0,LAA
4,026c97_0,4,026c97,0,CE
...,...,...,...,...,...
749,fe9645_0,3,fe9645,0,CE
750,fe9bec_0,4,fe9bec,0,LAA
751,ff14e0_0,6,ff14e0,0,CE
752,ffec5c_0,7,ffec5c,0,LAA


In [32]:
f_train_CE = f_train.loc[f_train['label'] == 'CE']['image_id']
f_train_LAA = f_train.loc[f_train['label'] == 'LAA']['image_id']

In [33]:
f_train_CE

0      006388_0
1      008e5c_0
4      026c97_0
6      029c68_0
7      032f10_0
         ...   
746    fdf998_0
747    fdf998_1
748    fe0cca_0
749    fe9645_0
751    ff14e0_0
Name: image_id, Length: 547, dtype: object

In [34]:
f_train_LAA

2      00c058_0
3      01adc5_0
5      028989_0
10     03d1ec_0
20     055f6a_0
         ...   
740    fd684f_0
745    fde9ba_0
750    fe9bec_0
752    ffec5c_0
753    ffec5c_1
Name: image_id, Length: 207, dtype: object

In [35]:
f_other = pd.read_csv('other.csv', sep=',')['image_id']
f_other

0      01f2b3_0
1      01f2b3_1
2      02ebd5_0
3      0412ab_0
4      04414e_0
         ...   
391    faaa7e_0
392    fd0f11_0
393    fd0f11_1
394    fd83c3_0
395    febb2b_0
Name: image_id, Length: 396, dtype: object

In [ ]:
Image.MAX_IMAGE_PIXELS = None
seam_carving.carve.MAX_MEAN_ENERGY = 10.0
crop_size = 256

def tif_to_png(name):
    img_tif = Image.open('Dataset/' + name + '.tif')
    size_tif = img_tif.size
    img_tif.thumbnail((size_tif[0]//2, size_tif[1]//2))
    img_tif.save('Dataset/' + name + '.png', "PNG", quality=100)
    
def crop_png(name, lbl_img):
    img = cv2.imread('Dataset/' + name + '.png')
    size = img.shape
    n_width = size[0]//crop_size
    n_height = size[1]//crop_size
    init_width = (size[0] - (n_width * crop_size))//2
    init_height = (size[1] - (n_height * crop_size))//2
    cnt = 0
    cnt_t = 0
    f_images = open('images.csv', 'a')
    w_images = csv.writer(f_images)
    for nw in range(n_width):
        for nh in range(n_height):
            img_crop = img[init_width+(nw*crop_size):init_width+((nw+1)*crop_size), init_height+(nh*crop_size):init_height+((nh+1)*crop_size), :]
            img_gray = cv2.cvtColor(img_crop, cv2.COLOR_BGR2GRAY)
            cp_w = (np.count_nonzero(img_gray > 200)/65536)*100
            cp_b = (np.count_nonzero(img_gray < 40)/65536)*100
            if (cp_w <= 20) and (cp_b <= 20):
                cnt_t += 1
                cv2.imwrite('Dataset_batches/' + name + '_' + str(cnt) + '.png', img_crop)
                if lbl_img == 'CE':
                    label_img = 1
                elif lbl_img == 'LAA':
                    label_img = 2
                else:
                    label_img = 0
                w_images.writerow([name + '_' + str(cnt) + '.png', str(label_img)])
            cnt += 1
    f_images.close()

    f_transf = open('transf.csv', 'a')
    w_transf = csv.writer(f_transf)
    w_transf.writerow([name, str(cnt_t)])
    f_transf.close()

**Descargar imágenes CE**

In [ ]:
for i in range(0, len(list(f_train_CE))):
    f = list(f_train_CE)[i]
    os.system('kaggle competitions download -f other/{:s}.tif mayo-clinic-strip-ai'.format(f))
    if os.path.exists('{:s}.tif.zip'.format(f)):
        os.system('unzip {:s}.tif.zip -d Dataset'.format(f))
        os.system('rm {:s}.tif.zip'.format(f))
    elif os.path.exists('{:s}.tif'.format(f)):
        os.system('mv {:s}.tif Dataset'.format(f))
    tif_to_png(f)
    os.system('rm Dataset/{:s}.tif'.format(f))
    crop_png(f, 'Other')
    os.system('rm Dataset/{:s}.png'.format(f))
    print('{:d} de {:d}'.format(i, len(f_other)))

**Descargar imágenes LAA**

In [ ]:
for i in range(0, len(list(f_train_LAA))):
    f = list(f_train_LAA)[i]
    os.system('kaggle competitions download -f other/{:s}.tif mayo-clinic-strip-ai'.format(f))
    if os.path.exists('{:s}.tif.zip'.format(f)):
        os.system('unzip {:s}.tif.zip -d Dataset'.format(f))
        os.system('rm {:s}.tif.zip'.format(f))
    elif os.path.exists('{:s}.tif'.format(f)):
        os.system('mv {:s}.tif Dataset'.format(f))
    tif_to_png(f)
    os.system('rm Dataset/{:s}.tif'.format(f))
    crop_png(f, 'Other')
    os.system('rm Dataset/{:s}.png'.format(f))
    print('{:d} de {:d}'.format(i, len(f_other)))

**Descargar imágenes Others**

In [ ]:
for i in range(0, len(list(f_other))):
    f = list(f_other)[i]
    os.system('kaggle competitions download -f other/{:s}.tif mayo-clinic-strip-ai'.format(f))
    if os.path.exists('{:s}.tif.zip'.format(f)):
        os.system('unzip {:s}.tif.zip -d Dataset'.format(f))
        os.system('rm {:s}.tif.zip'.format(f))
    elif os.path.exists('{:s}.tif'.format(f)):
        os.system('mv {:s}.tif Dataset'.format(f))
    tif_to_png(f)
    os.system('rm Dataset/{:s}.tif'.format(f))
    crop_png(f, 'Other')
    os.system('rm Dataset/{:s}.png'.format(f))
    print('{:d} de {:d}'.format(i, len(f_other)))

In [ ]:
lst_imgs = os.listdir('Dataset_batches')
lst_imgs.sort()

In [ ]:
f_images = open('images.csv', 'w')
w_images = csv.writer(f_images)
w_images.writerow(['image', 'label'])
cnt_CE = 0
cnt_LAA = 0
cnt_oth = 0
lst_CE = []
lst_LAA = []
lst_oth = []
for i in range(len(lst_imgs)):
    if lst_imgs[i][0:8] in list(f_train_CE):
        w_images.writerow([lst_imgs[i], '1'])
        cnt_CE += 1
        lst_CE.append(lst_imgs[i][0:8])
    elif lst_imgs[i][0:8] in list(f_train_LAA):
        w_images.writerow([lst_imgs[i], '2'])
        cnt_LAA += 1
        lst_LAA.append(lst_imgs[i][0:8])
    elif lst_imgs[i][0:8] in list(f_other):
        w_images.writerow([lst_imgs[i], '0'])
        cnt_oth += 1
        lst_oth.append(lst_imgs[i][0:8])
f_images.close()

In [36]:
lst_CE = list(set(lst_CE))
lst_LAA = list(set(lst_LAA))
lst_oth = list(set(lst_oth))
lst_CE.sort()
lst_LAA.sort()
lst_oth.sort()

print('CE => pasar de {:d} a {:d}'.format(len(lst_CE), cnt_CE))
print('LAA => pasar de {:d} a {:d}'.format(len(lst_LAA), cnt_LAA))
print('Others => pasar de {:d} a {:d}'.format(len(lst_oth), cnt_oth))

CE => pasar de 116 a 28182
LAA => pasar de 71 a 20122
Others => pasar de 59 a 13278


In [ ]:
df = pd.read_csv('images.csv')
df

In [ ]:
imgs = []
for i in list(df['image']):
    img = cv2.imread('Dataset_batches/' + i)
    imgs.append(img)

In [ ]:
imgs = np.array(imgs)
imgs = imgs.astype('float32')
imgs /= 255
np.save('imgs.npy', imgs)
lbls = np.array(df['label'])
np.save('lbls.npy', lbls)

<hr>

### Rediseño del dataset

1.000 imágenes por categoría
3.000 en total

Se toman del archivo respectivo 'images.csv'

Imágenes aleatorias, para que no sean del mismo coágulo sino de diversos para poder entrenar bien

Se van a mover a la carpeta 'Dataset' que antes se usaba como transitoria entre las que se descargaban completas y los recortes

In [2]:
df = pd.read_csv('images.csv')
df

,image,label
0,006388_0_1002.png,1
1,006388_0_1006.png,1
2,006388_0_1007.png,1
3,006388_0_1008.png,1
4,006388_0_1009.png,1
...,...,...
61577,69d655_0_745.png,2
61578,69d655_0_784.png,2
61579,69d655_0_788.png,2
61580,69d655_0_96.png,2


In [11]:
lst_CE = list(df.loc[df['label'] == 1]['image'])

In [14]:
random.shuffle(lst_CE)
lst_CE

['288156_0_7504.png',
 '3020ef_0_4051.png',
 '0d533f_0_2737.png',
 '2244d2_0_1619.png',
 '04f7a4_0_1607.png',
 '1d1bf0_0_257.png',
 '2db520_0_3782.png',
 '2244d2_0_1113.png',
 '31adaa_1_471.png',
 '2db520_0_9374.png',
 '1d1bf0_0_347.png',
 '140d1a_0_1701.png',
 '288156_0_3884.png',
 '04439c_0_530.png',
 '09644e_4_804.png',
 '0e696a_0_1787.png',
 '251bfc_0_944.png',
 '006388_0_358.png',
 '09644e_3_997.png',
 '13f372_0_3345.png',
 '27afc7_0_386.png',
 '0e4dac_0_6587.png',
 '0415c3_0_1974.png',
 '1ec69e_0_1983.png',
 '0415c3_0_8053.png',
 '34c9bc_0_620.png',
 '2db520_0_13058.png',
 '0415c3_0_7343.png',
 '006388_0_4657.png',
 '0415c3_0_6575.png',
 '19b036_0_225.png',
 '2244d2_0_257.png',
 '217b27_0_1799.png',
 '2244d2_0_5372.png',
 '217b27_0_1829.png',
 '1ef6da_0_2851.png',
 '14d2fa_0_1715.png',
 '24f8f5_0_2424.png',
 '23d2c1_1_4926.png',
 '140d1a_0_169.png',
 '34c9bc_0_1701.png',
 '0e4dac_0_7536.png',
 '1f9d4f_0_4516.png',
 '0e4dac_0_1392.png',
 '13f372_0_2739.png',
 '12c8c9_0_1628.png',


In [15]:
indexes = random.sample(range(0, len(lst_CE)), 1000)

In [18]:
lst_CE_ = []
for i in indexes:
    lst_CE_.append(lst_CE[i])

In [19]:
lst_CE_

['217b27_0_247.png',
 '006388_0_7157.png',
 '0d4164_0_5493.png',
 '0d4164_0_4121.png',
 '0bddf9_0_3358.png',
 '2244d2_0_5083.png',
 '2db520_0_9098.png',
 '0415c3_0_2200.png',
 '2db520_0_3704.png',
 '13f372_0_2759.png',
 '20914b_0_1222.png',
 '0d533f_0_2959.png',
 '1f9d4f_0_5445.png',
 '3020ef_0_2815.png',
 '194e3f_0_1086.png',
 '2db520_0_8939.png',
 '006388_0_4120.png',
 '0aaeb3_0_2083.png',
 '282d0a_0_239.png',
 '2e7d16_0_1615.png',
 '1ef6da_0_698.png',
 '24f8f5_0_2675.png',
 '282d0a_0_218.png',
 '2e7d16_0_6404.png',
 '0e4dac_0_1718.png',
 '029c68_0_2007.png',
 '33d2a7_0_2251.png',
 '0d533f_0_2435.png',
 '13f372_0_4520.png',
 '006388_0_5784.png',
 '0d718a_0_609.png',
 '0e4dac_0_5309.png',
 '33d071_0_1511.png',
 '09644e_3_600.png',
 '1f9d4f_0_10242.png',
 '13f372_0_5626.png',
 '3020ef_0_5014.png',
 '0415c3_0_8056.png',
 '0c60b8_0_1248.png',
 '05a1ec_0_2387.png',
 '19dd95_0_197.png',
 '1f9d4f_0_10175.png',
 '2745fc_0_2694.png',
 '3020ef_0_8220.png',
 '0bddf9_0_4993.png',
 '1f9d4f_0_9186

In [20]:
len(lst_CE_)

1000

In [21]:
for i in range(1000):
    os.system('mv Dataset_batches/{:s} Dataset/'.format(lst_CE_[i]))

In [22]:
lst_LAA = list(df.loc[df['label'] == 2]['image'])
random.shuffle(lst_LAA)
indexes = random.sample(range(0, len(lst_LAA)), 1000)
lst_LAA_ = []
for i in indexes:
    lst_LAA_.append(lst_LAA[i])
print(len(lst_LAA_))
for i in range(1000):
    os.system('mv Dataset_batches/{:s} Dataset/'.format(lst_LAA_[i]))

1000


In [23]:
lst_others = list(df.loc[df['label'] == 0]['image'])
random.shuffle(lst_others)
indexes = random.sample(range(0, len(lst_others)), 1000)
lst_others_ = []
for i in indexes:
    lst_others_.append(lst_others[i])
print(len(lst_others_))
for i in range(1000):
    os.system('mv Dataset_batches/{:s} Dataset/'.format(lst_others_[i]))

1000


Leer las listas individuales para montarlas en un dataFrame para posteriormente guardarlo en un archivo para su posterior apertura en el notebook necesario

In [24]:
df_imgs = []
for i in range(1000):
    df_imgs.append([lst_others_[i], 0])
    df_imgs.append([lst_CE_[i], 1])
    df_imgs.append([lst_LAA_[i], 2])
df_imgs = pd.DataFrame(df_imgs, columns=['image', 'label'])
df_imgs

,image,label
0,22ee77_0_2101.png,0
1,217b27_0_247.png,1
2,2b7304_1_669.png,2
3,151856_0_437.png,0
4,006388_0_7157.png,1
...,...,...
2995,291099_0_1697.png,1
2996,41b23c_0_2159.png,2
2997,283ec7_0_5601.png,0
2998,2eb2d7_0_599.png,1


In [25]:
df_imgs.to_csv('images_labels_1000.csv')